# Custom Dot Detection by Threshold Optimization

In [ ]:
#custom function
from dot_detection_blob_rad_gaus import *

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        name = "Gaussian"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "LoG"
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)

In [ ]:
# #test find threshold parameters
# final = find_threshold(img_src, min_sigma = 1.5, max_sigma = 5, 
#                    num_sigma = 5, threshold_min = 0.0005, 
#                    threshold_max= 0.02, interval=200, 
#                    HybCycle = 0, channel = 0, min_dots_start=10000, 
#                    min_dots_end = 100000,pos_start=0,pos_end=1,reduce_cutoff= 2, window=10, strict=False)

In [ ]:
import time
#start time
start = time.time()

#image sources
img_src = "/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/deconvoluted_images/HybCycle_0/MMStack_Pos0.ome.tif"
img_raw = '/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dapi_aligned_gaussian/fiducial_aligned/HybCycle_0/MMStack_Pos0.ome.tif'
raw_src=None

#img_src: path to image
#min_sigma: parameter for blob log
#max_sigma: parameter for blob log
#num_sigma:parameter for blob log
#channel: which channel you want to analyze
#pos: position number if check initial is True
#choose_thresh_set:which threshold set you want to use (if you have already ran full screen)
#hyb_number:total number of hybs for choose thresh set(if you are picking one)
#check_initial: check initial optimal threshold
#region_size: bounding box size for gaussian fit or radial centering
#gaussian: bool for performing gaussian fit
#radial_center: bool for radial centering
#optimize: bool to test different threshold and min dots
#output: bool to write out results or return data

dots = dot_detection(img_src, min_sigma = 1.5, max_sigma = 5, 
                     num_sigma = 5, HybCycle=0, size_cutoff=2, 
                     opt_thresh=0.001,channel=0,pos=0,choose_thresh_set = 6, hyb_number=80,
                     check_initial = False, region_size=5,
                     gaussian = True,radial_center=False, optimize=True, output=False)

print(f"This task took {(time.time() - start)/60} minutes")

In [ ]:
# dots

In [ ]:
#from dotdetection opt
thresh = 4
dots = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_comb/Channel_1_radialcenter/MMStack_Pos0/Threshold_{thresh}/Dot_Locations/locations_z_0.csv")

In [ ]:
#isolate hyb
hyb = 0
check = dots[dots["hyb"]==hyb]

In [ ]:
img_raw = '/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dapi_aligned/fiducial_aligned_gaussian/HybCycle_0/MMStack_Pos0.ome.tif'
plot_3d_locs_on_2d_image(check,None, tiff=None, channel=0, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=2000)

In [ ]:
tiff = tf.imread(img_src)
plot_3d_locs_on_2d_image(check,None, tiff=tiff, channel=0, 
                         raw_src = img_raw, raw_image = False,add_trace=False, zmax=1000)

# Organize dots detected files

In [1]:
#custom functions
from dot_detection_blob_rad_gaus import combine_dot_files

In [3]:
path_dots = "/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_detected/Channel_1_radial"
combine_dot_files(path_dots, hyb_start=0,hyb_end=80,num_HybCycle = 80,
                  pos = 0, channel=0, num_z = 2, opt_files = True)